In [ ]:
'''
掛載Google硬碟
安裝套件
引用套件
APP應用準備
消息準備
handler執行方法設計
啟動應用

'''

In [ ]:
'''

資料 mapping 至google drive

把資料寫在/content/drive

即可保存在 google drive內

'''
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
'''

流程解析

'''


In [ ]:
!pip install line-bot-sdk flask flask-ngrok

In [ ]:
'''
引用套件
'''

# 引用Web Server套件
from flask import Flask, request, abort, jsonify

# 載入json處理套件
import json

# 外部連結自動生成套件
from flask_ngrok import run_with_ngrok

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)


In [ ]:
'''
建置主程序

建置handler與 line_bot_api
'''

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

# 生成實體物件
line_bot_api = LineBotApi("CHANNEL_ACCESS_TOKEN")
handler = WebhookHandler("CHANNEL_SECRET")

In [ ]:
'''
建置主程序的API入口
  接受Line傳過來的消息
  並取出消息內容
  將消息內容存在google drive的檔案內
  並請handler 進行消息驗證與轉發
'''


# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print(body)

    # 記錄用戶log
    f = open("/content/drive/MyDrive/ai-event.log", "a")
    f.write(body)
    f.close()

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [ ]:
'''

準備QuickReply的Button


'''

# 引入相關套件
from linebot.models import (
    MessageAction, URIAction,
    PostbackAction, DatetimePickerAction,
    CameraAction, CameraRollAction, LocationAction,
    QuickReply, QuickReplyButton
)

# 創建QuickReplyButton 

## 點擊後，以用戶身份發送文字消息
## MessageAction
textQuickReplyButton = QuickReplyButton(
    action=MessageAction(
        label="發送文字消息", 
        text="text2"
    )
)

## 點擊後，彈跳出選擇時間之視窗
## DatetimePickerAction
dateQuickReplyButton = QuickReplyButton(
    action=DatetimePickerAction(
        label="時間選擇",
        data="data3",                       
        mode="date"
    )
)


## 點擊後，開啟相機
## CameraAction
cameraQuickReplyButton = QuickReplyButton(
    action=CameraAction(label="拍照")
)

## 點擊後，切換至照片相簿選擇
cameraRollQRB = QuickReplyButton(
    action=CameraRollAction(label="選擇照片")
)

## 點擊後，跳出地理位置
locationQRB = QuickReplyButton(
    action=LocationAction(label="地理位置")
)

## 點擊後，以Postback事件回應Server 
postbackQRB = QuickReplyButton(
    action=PostbackAction(label="我是PostbackEvent", data="Data1")
)

In [ ]:
'''

以QuickReply封裝該些QuickReply Button

'''
## 設計QuickReplyButton的List
quickReplyList = QuickReply(
    items = [textQuickReplyButton, dateQuickReplyButton, cameraQuickReplyButton, cameraRollQRB, locationQRB,postbackQRB]
)

In [ ]:
'''

製作TextSendMessage，並將剛封裝的QuickReply放入

'''
## 將quickReplyList 塞入TextSendMessage 中 
from linebot.models import (
    TextSendMessage,
)
quick_reply_text_send_message = TextSendMessage(text='發送問題給用戶，請用戶回答', quick_reply=quickReplyList)

In [ ]:
'''

設計一個字典
    當用戶輸入相應文字消息時，系統會從此挑揀消息

'''

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

# 根據自定義菜單四張故事線的圖，設定相對應image
template_message_dict = {
  "@reply":quick_reply_text_send_message,
}

In [ ]:
#用戶點擊button後，觸發postback event，對其回傳做相對應處理
from linebot.models import PostbackEvent

@handler.add(PostbackEvent)
def handle_post_message(event):
    user_profile = line_bot_api.get_profile(event.source.user_id)
    if (event.postback.data.find('Data1')== 0):
        with open("user_profile_business.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextSendMessage(
                    text='請稍待，會有專人與您聯繫'
                )
            )
    elif (event.postback.data.find('Data2') == 0):
        with open("user_profile_tutorial.txt", "a") as myfile:
            myfile.write(json.dumps(vars(user_profile),sort_keys=True))
            myfile.write('\n')
            line_bot_api.reply_message(
            event.reply_token,
                TextSendMessage(
                    text='請稍待，我們會派專家您聯繫'
                )
            )
    else:
        pass

In [ ]:
# 用戶發出文字消息時， 按條件內容, 回傳文字消息
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    
    if(event.message.text.find('@')!= -1):
        line_bot_api.reply_message(
        event.reply_token,
        template_message_dict.get(event.message.text)
        )
    else:
        line_bot_api.reply_message(
        event.reply_token,
        TextSendMessage(text="字典內查無此字，請輸入@more")
        )

In [ ]:
# 主程序運行
app.run()